# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries

# Load data
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

# NLP pipelines
import nltk
import re

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


# ML Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split

#sklearn classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

# 5 test your model
from sklearn.metrics import classification_report

# 6 Improve your model
from sklearn.model_selection import GridSearchCV

# 9. Export your model as a pickle file
from datetime import date, datetime
from pytz import timezone
import pickle

from workspace_utils import active_session
import sys

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
def load_data(db):
    """
    INPUT:
        A database file name like: 'disasterResponse.db'
    
    OUTPUT:
        
        Feature and target variables X and Y
        
        X = df.message.values
        Y =  df[df.columns[4:]].values
        
    """
    engine = create_engine(f'sqlite:///{db}')
    df = pd.read_sql('SELECT * FROM disasterResponse', engine)
    X = df.message.values # Feature
    Y =  df[df.columns[4:]].values # Target
    
    return X, Y, df

In [3]:
X, Y, df = load_data("disasterResponse_18_09_2019_05_01_50.db")

In [4]:
X[:5]

array(['Is the Hurricane over or is it not over',
       'UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately.',
       'Storm at sacred heart of jesus',
       'Please, we need tents and water. We are in Silo, Thank you!',
       'I am in Croix-des-Bouquets. We have health issues. They ( workers ) are in Santo 15. ( an area in Croix-des-Bouquets )'], dtype=object)

In [5]:
Y[:3]

array([[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]])

In [6]:
df.shape

(14785, 38)

### 2. Write a tokenization function to process your text data

In [7]:
def tokenize(text):
    # Normalize 
    text = text.lower().strip() # Tudo minusculo 
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)

    # Tokenize text 
    words = word_tokenize(text) # Separa as sentenças por palavras

    # Testar com e sem stop words
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")] # Remove palavras padrão

    # lemmatizer root words
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in words:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip() # Root words
        clean_tokens.append(clean_tok)
        
    return clean_tokens


In [8]:
teste = tokenize(X[3])
print(teste)

['please', 'need', 'tent', 'water', 'silo', 'thank']


### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [9]:
# https://stackoverflow.com/questions/46770088/multioutput-classifier-learning-5-target-variables

In [10]:
lm  = RandomForestClassifier() # 5 min
pipeline = Pipeline(
    [("vect", CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ("clf", MultiOutputClassifier(lm)),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
def what_time():
    """
    Return the time from "America/Sao_Paulo"
    
    """
    time_now =  datetime.now(timezone("America/Sao_Paulo")).strftime("%H:%M:%S")
    
    return time_now

In [12]:
# perform train test split
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=.33, random_state=42)

In [13]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=5, test_size=0.5, random_state=42)

for train_index, test_index in sss.split(X, Y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.

In [ ]:
what_time()

In [ ]:
# train classifier
pipeline.fit(X_train, y_train)

In [ ]:
# predict on test data
y_pred = pipeline.predict(X_test)

In [ ]:
what_time()

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [ ]:
import warnings

def fxn():
    warnings.warn("UndefinedMetricWarning", DeprecationWarning)

In [ ]:
# display results
# https://pt.stackoverflow.com/questions/309420/classification-report-e-confusion-matrix-do-sklearn-os-valores-n%C3%A3o-batem
def display_results(y_test, y_pred, target_names):
    """
    INPUT:
        y_teste = Target test
        y_pred = Prediction target
        target_names = Columns names of targets.
    
    OUTPUT:
        Print: f1 score, precision and recall 
    """
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        fxn()
        
        for i in range(len(target_names)):
            print(target_names[i])
            print(classification_report(y_test[:, i], y_pred[:, i]))

In [ ]:
target_names = df.columns[4:]
display_results(y_test, y_pred, target_names)

In [ ]:
print(pipeline)

In [ ]:
what_time()

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
   parameters_0 = {
        'vect__ngram_range': ((1, 1), (1, 2)),
        'vect__max_df': (0.5, 0.75, 1.0),
        'vect__min_df': (0.1, 0.05),
        'vect__max_features': (None, 100, 500, 1000, 5000),
        
        
        'clf__estimator__n_estimators': [50, 100, 200],
        'clf__estimator__max_depth': [4, 8, 16],
        'clf__estimator__min_samples_leaf': [2, 4, 8],
        'clf__estimator__min_samples_split': [2, 4, 8],
    
    }

In [ ]:
parameters_1 = {
    #'clf__estimator__n_estimators': [100, 200],
    'clf__estimator__min_samples_split': [2, 3, 4],

    #'clf__estimator__max_depth': [4, 8, 16],
    #'clf__estimator__min_samples_leaf': [2, 4, 8],

}

In [ ]:
def build_model():
    """
    Returns a model
    """
    
    # Build a machine learning pipeline
    
    lm  = RandomForestClassifier()
    
    pipeline = Pipeline(
    [("vect", CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ("clf", MultiOutputClassifier(lm)),
    ])
    
     # specify parameters for grid search
#     parameters = {
#     'clf__estimator__min_samples_split': [2, 3, 4]
#     }
    
    parameters = {
        #'clf__estimator__n_estimators': [100, 200],
        'clf__estimator__min_samples_split': [2, 3, 4],

        #'clf__estimator__max_depth': [4, 8, 16],
        #'clf__estimator__min_samples_leaf': [2, 4, 8],
    }
    
    # create grid search object
    #par = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
what_time()

In [ ]:
model =  build_model()

In [ ]:
model

In [ ]:
model.estimator.get_params().keys()

In [ ]:
what_time() 

In [ ]:
# Keeping Your Session Active
from workspace_utils import active_session
with active_session():
    model.fit(X_train, y_train)

what_time()

In [ ]:
with active_session():
    y_pred = model.predict(X_test)

In [ ]:
display_results(y_test, y_pred, target_names)

In [ ]:
what_time()

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# https://github.com/gilmar/udacity-live-coding-ensembles/blob/master/udacity-live-coding-ensembles.ipynb
import warnings
def evaluate_classifiers(models, X_train, y_train,  X_test, y_test, target_names):
    results = []
    for name in models.keys():
        try:
            lm = models[name]
            
            print(lm)

            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                fxn()

                model = Pipeline(
                    [("vect", CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ("clf", MultiOutputClassifier(lm)),])

                model.fit(X_train, y_train)
                y_pred = model.predict(X_test)


                #acc = accuracy_score(y_test, y_pred)

                #cm = confusion_matrix(y_test, y_pred)
                
                for i in range(len(target_names)):
                    print("{} \n {}".format(target_names[i], classification_report(y_test[:, i], y_pred[:, i])))
                    report = "{} \n {}".format(target_names[i], classification_report(y_test[:, i], y_pred[:, i]))

                #report = classification_report(y_test, y_pred)
                print(report)


                results.append((name, report))
        except: 
            print("erro {}".format(models[name]))
            print(sys.exc_info())
            pass
    return results

In [ ]:
# def report_classifiers_results(model_results, metrics=['acc', 'cm', 'report']):
#     for result in model_results:
#         name, report = result
#         print("Model: %s" % name)
#         #if metrics.contains('acc'): print("Accuracy: %f" % acc) 
#         #if metrics.contains('cm'): print("Confusion Matrix: \n %s" % cm)
#         print(report)
#         print()

In [ ]:
models = {}
models['LR'] = LogisticRegression()
models['LDA'] = LinearDiscriminantAnalysis()
models['KNN'] = KNeighborsClassifier()
models['CART'] = DecisionTreeClassifier(random_state=13)
models['NB'] = GaussianNB()
models['SVM'] = SVC()
# ===
models["RF"] = RandomForestClassifier()
#models["DUM"] = DummyClassifier()
#models["BAG"] = BaggingClassifier()
models["ADB"] = AdaBoostClassifier()
#models["VOT"] = VotingClassifier()

In [ ]:
what_time()

In [ ]:
with active_session():
    classifiers_results = evaluate_classifiers(models, X_train, y_train, X_test, y_test, target_names)

In [ ]:
what_time()

In [ ]:
def build_tunning_model():
    """
    Returns a model
    """
    
    # Build a machine learning pipeline
    
    lm  = LogisticRegression()
    
    pipeline = Pipeline(
    [("vect", CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ("clf", MultiOutputClassifier(lm)),
    ])
    
     # specify parameters for grid search
    parameters = {
    'clf__estimator__C': [1, 10, 100, 1000],
    'clf__estimator__multi_class' : ["ovr", "multinomial", "auto"]
    }
    
    # create grid search object
    #par = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
    cv = GridSearchCV(pipeline, param_grid=parameters)
    
    return cv

In [ ]:
model_2 = build_tunning_model()

In [ ]:
model_2.estimator.get_params().keys()

In [ ]:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [ ]:
parameters = {
    'clf__estimator__C': [1, 10, 100, 1000],
    'clf__estimator__multi_class' : [‘ovr’, ‘multinomial’, ‘auto’]
}

In [ ]:
parameters = {
    'clf__estimator__min_samples_split': [2, 3, 4]
    }

In [ ]:
# Keeping Your Session Active
from workspace_utils import active_session
with active_session():
    model_2.fit(X_train, y_train)

what_time()

In [ ]:
with active_session():
    y_pred = model_2.predict(X_test)

In [ ]:

what_time()

### 9. Export your model as a pickle file

In [ ]:
# https://machinelearningmastery.com/save-load-machine-learning-models-python-scikit-learn/

In [ ]:
# datetime object containing current date and time
# https://www.programiz.com/python-programming/datetime/current-datetime
now = datetime.now()
dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")

# save the model to disk
filename = f'model_{dt_string}.sav'
pickle.dump(pipeline, open(filename, 'wb'))

In [ ]:
def export_model(model):
    """
    Save your model as a pikle file in current local directory.
    
    The output file looks like "model_10_09_2019_02_21_04.sav". 
    
    Where the numbers indicate the date and the time that file was create - %d_%m_%Y_%H_%M_%S".
    """
    
    now = datetime.now()
    dt_string = now.strftime("%d_%m_%Y_%H_%M_%S")

    # save the model to disk
    filename = f'model_{dt_string}.sav'
    pickle.dump(model, open(filename, 'wb'))
    

In [ ]:
export_model(model)

In [ ]:
# load the model from disk
filename = "model_10_09_2019_02_21_04.sav"

loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)

print(result)

In [ ]:
def load_model(file_model):
    """
    load the model from disk:
    
    INPUT:
        A pickle file model name.
    
    OUTPUT:
        A ML model
    """ 
    loaded_model = pickle.load(open(filename, 'rb'))
    return  loaded_model 

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.